Importación y Carga de Datos

In [106]:
# Importaciones
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from xgboost import XGBClassifier
import keras_tuner as kt
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

df = pd.read_csv('telecom_customer_churn.csv')

Preprocesamiento

In [107]:
# 1. Preprocesamiento
# a. Eliminar entradas con el estado "Joined"
df = df[df['Customer Status'] != 'Joined']

# b. Crear la columna 'Churn' y asignar 1 si 'Customer Status' es 'Churned', de lo contrario 0
df['Churn'] = df['Customer Status'].apply(lambda x: 1 if x == 'Churned' else 0)

# c. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# d. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# e. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Customer Status':
        # Llenar valores faltantes con la moda para columnas categóricas
        df[column].fillna(df[column].mode()[0], inplace=True)
        # Convertir variables categóricas a numéricas
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        # Llenar valores faltantes con la mediana para columnas numéricas
        df[column].fillna(df[column].median(), inplace=True)

# Eliminar la columna 'Customer Status' antes de normalizar
df = df.drop(columns=['Customer Status'])

# f. Normalización
# Seleccionar solo las columnas que no son 'Churn' para la normalización
cols_to_scale = df.columns.difference(['Churn'])
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

Selección de características Gradient Boosting

In [108]:
# 2. Selección de características
xgb_for_feature_selection = XGBClassifier(
    objective='binary:logistic', 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)
xgb_for_feature_selection.fit(df.drop('Churn', axis=1), df['Churn'])
threshold = 0.01  
selected_features = df.drop('Churn', axis=1).columns[(xgb_for_feature_selection.feature_importances_ > threshold)].tolist()

Balanceo con SMOTE

In [109]:
# 3. Balanceo con SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(df[selected_features], df['Churn'])

División de ConjuntoBalanceo

In [110]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

Entrenamiento de ANN

In [111]:
# 5. Entrenamiento de ANN con búsqueda de hiperparámetros usando Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=64, step=8), 
                    input_dim=len(selected_features), activation='relu'))
    model.add(Dense(units=hp.Int('units', min_value=8, max_value=64, step=8), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=100,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='keras_tuner_example'
)

# Aplicar Early Stopping y Model Checkpoint como callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Ejecutar la búsqueda de hiperparámetros
tuner.search(X_train, y_train, epochs=100, validation_split=0.2, callbacks=callbacks)

# Obtener los mejores hiperparámetros.
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Imprimir los mejores hiperparámetros
print(f"""
Los mejores hiperparámetros encontrados:
Número de unidades: {best_hps.get('units')}
Tasa de aprendizaje: {best_hps.get('learning_rate')}
""")

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

Trial 23 Complete [00h 00m 14s]
val_accuracy: 0.8199867606163025

Best val_accuracy So Far: 0.8272666931152344
Total elapsed time: 00h 04m 52s

Los mejores hiperparámetros encontrados:
Número de unidades: 48
Tasa de aprendizaje: 0.001



Evaluación del modelo

In [112]:
# 6. Evaluación del modelo con los mejores hiperparámetros
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Obtener las predicciones en forma de clases
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")

# Calcular la matriz de confusión, y generar el reporte de clasificación
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Imprimir las métricas
print(conf_matrix)
print("Reporte de clasificación:\n", report)

59/59 [==============================] - 2s 7ms/step - loss: 0.4028 - accuracy: 0.8279
Loss: 0.4027519226074219, Accuracy: 0.8278601765632629
59/59 [==============================] - 0s 2ms/step
[[764 190]
 [135 799]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.85      0.80      0.82       954
           1       0.81      0.86      0.83       934

    accuracy                           0.83      1888
   macro avg       0.83      0.83      0.83      1888
weighted avg       0.83      0.83      0.83      1888



Validación cruzada

In [113]:
# 7. Validación Cruzada
def create_best_model():
    model = Sequential()
    model.add(Dense(units=best_hps.get('units'), input_dim=len(selected_features), activation='relu'))
    model.add(Dense(units=best_hps.get('units'), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=best_hps.get('learning_rate'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Envolver el modelo en un KerasClassifier
best_model_sklearn = KerasClassifier(build_fn=create_best_model, epochs=100, batch_size=10, verbose=0)

# Realizar la validación cruzada
scores = cross_val_score(best_model_sklearn, X_train, y_train, cv=5, scoring='accuracy')

# Imprimir el promedio de los scores de la validación cruzada
print("Accuracy promedio con Validación Cruzada:", scores.mean())

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a 

Accuracy promedio con Validación Cruzada: 0.647647757504569
